In [4]:
# T-A: Head-only fine-tuning (best-by macro-F1)
from pathlib import Path
import json, numpy as np, torch
from torch import nn, optim
from torch.utils.data import Subset, DataLoader
from torchvision import datasets, transforms, models
from sklearn.metrics import f1_score, confusion_matrix

# project paths (adjust if needed)
ROOT = Path(__file__).resolve().parent if "__file__" in globals() else Path.cwd()
DATA_ROOT = ROOT / "asl_alphabet_train"
ART = ROOT / "artifacts"
CKPT = ROOT / "checkpoints"; CKPT.mkdir(exist_ok=True)
RESULTS = ROOT / "results"; RESULTS.mkdir(exist_ok=True)

SEED = 429
BATCH_SIZE = 64
IMG_SIZE = 224
NUM_WORKERS = 2

torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)

Device: cpu
